### Collab Notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install xgboost

In [ ]:
from xgboost.sklearn import XGBClassifier

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, SimpleRNN, Flatten,\
Activation, RepeatVector, Permute, merge, Lambda
# from keras_contrib.layers import CRF
import keras.optimizers as ko
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
import keras
import subprocess
from sklearn.utils import class_weight
import keras.backend as K
from keras.layers.convolutional import Conv1D, MaxPooling1D
PATH = 'drive/My Drive/Seq_Classification/'

In [ ]:
data = pd.read_csv(PATH+'task2_typ.csv')
data = data.rename(columns={'id':'Sentence #'})
data = data.dropna(axis=0)
# data = data.drop('Unnamed: 0',axis=1)

In [ ]:
sentences = []
labels = list(data['type']) 
typs = data['types'].values

x = data.apply(lambda l: sentences.append(l['sentence'].split(' ')),axis=1)
words = []
for sent in sentences:
    for wrd in sent:
        words.append(wrd)
words = list(set(words))
tags = list(set(labels))

n_words = len(words)
n_tags = len(tags)
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
max_len = 30
X = [[word2idx[w] for w in s] for s in sentences]    
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [tag2idx[tg] for tg in labels]
# y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = np.array([to_categorical(i, num_classes=n_tags) for i in y])
y = [[y[i],typs[i]] for i in range(len(y))]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = np.array([lab[0] for lab in y_train])
typ_test = [lab[1] for lab in y_test]
y_test = np.array([lab[0] for lab  in y_test])

In [ ]:
def loadGloveModel(File):
    print("Loading Glove Model")
    f = open(File,'r')
    gloveModel = {}
    for line in f:
        splitLines = line.split()
        word = splitLines[0]
        wordEmbedding = np.array([float(value) for value in splitLines[1:]])
        gloveModel[word] = wordEmbedding
    print(len(gloveModel)," words loaded!")
    return gloveModel

vec_model = loadGloveModel(PATH+'glove/glove.6B.200d.txt')

Loading Glove Model
400000  words loaded!


In [ ]:
tag2idx = {t: i for i, t in enumerate(tags)}
max_len = 30
X = []
y = []
emb_dim = len(vec_model['the'])
for sent in sentences:
    vec = np.zeros((30,emb_dim))
    for i, word in enumerate(sent):
        if word in vec_model:
            vec[i,:] = vec_model[word]
    X.append(vec)
X = np.array(X)
y = np.array([tag2idx[tg] for tg in labels])

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
X_train = X_train.reshape((X_train.shape[0],max_len*emb_dim))
X_test = X_test.reshape((X_test.shape[0],max_len*emb_dim))

In [ ]:
clf = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27
)

In [ ]:
clf.fit(X_train,y_train)
pred = clf.predict(X_test)

In [ ]:
inc_test = []
exc_test = []
inc_pred = []
exc_pred = []
for i,t in enumerate(typ_test):
    if t == 'I':
        inc_test.append(y_test[i])
        inc_pred.append(pred[i])
    else:
        exc_test.append(y_test[i])
        exc_pred.append(pred[i])
print("For inclusions")
report = classification_report(inc_test,inc_pred,output_dict=True)
df = pd.DataFrame(report).transpose()
display(df)
print("For exclusions")
report = classification_report(exc_test,exc_pred,output_dict=True)
df = pd.DataFrame(report).transpose()
display(df)